# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [114]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [71]:
weather_data = os.path.join('..', 'Output', 'output_data_file.csv')

city_weather = pd.read_csv(weather_data)

city_weather.head()


,Unnamed: 0,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
0,0,Neuquén,AR,46.40,81,0,8.05,-38.9516,-68.0591,1618632636
1,1,Hobart,AU,63.00,51,75,14.97,-42.8794,147.3294,1618632569
2,2,Dikson,RU,-2.63,88,99,5.30,73.5069,80.5464,1618632511
3,3,East London,ZA,55.40,71,0,6.91,-33.0153,27.9116,1618632337
4,4,Karaikudi,IN,86.74,64,70,6.73,10.0667,78.7833,1618632636


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [72]:
gmaps.configure(g_key)

In [86]:
city_latlong= city_weather[["Latitude", "Longitude"]]


humidity_map = gmaps.figure(map_type="TERRAIN")


heat_layer = gmaps.heatmap_layer(city_latlong, weights=city_weather["Humidity"].astype(float), 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


humidity_map.add_layer(heat_layer)


humidity_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [91]:
ideal_weather = city_weather.loc[(city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80) & (city_weather["Cloudiness"] == 0), :]
ideal_weather = ideal_weather.dropna(how='any')
ideal_weather.reset_index(inplace=True)

del ideal_weather["Unnamed: 0"]
del ideal_weather["index"]


ideal_weather

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
0,Umluj,SA,76.91,67,0,3.40,25.0213,37.2685,1618632662
1,Bahía Honda,CU,71.98,78,0,4.61,22.9036,-83.1592,1618632501
2,Marsá Maţrūḩ,EG,71.60,35,0,11.50,31.3525,27.2453,1618632441
3,Yulara,AU,71.60,28,0,17.27,-25.2406,130.9889,1618632707
4,Atar,MR,70.54,31,0,9.55,20.5169,-13.0499,1618632740
5,Cabañas,CU,73.40,83,0,3.44,22.9758,-82.9219,1618632741
6,Manama,BH,77.00,41,0,10.36,26.2154,50.5832,1618632631
7,Alice Springs,AU,73.99,28,0,14.97,-23.7000,133.8833,1618632560


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [92]:
hotel_df = pd.DataFrame(ideal_weather, columns=["City", "Country", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Umluj,SA,25.0213,37.2685,
1,Bahía Honda,CU,22.9036,-83.1592,
2,Marsá Maţrūḩ,EG,31.3525,27.2453,
3,Yulara,AU,-25.2406,130.9889,
4,Atar,MR,20.5169,-13.0499,
5,Cabañas,CU,22.9758,-82.9219,
6,Manama,BH,26.2154,50.5832,
7,Alice Springs,AU,-23.7000,133.8833,


In [111]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        time.sleep(1)
    except (KeyError, IndexError):
        print("Could not find a hotel within 5000 meters")
        
        
print("--------------------n/Search Complete/n--------------------")

Retrieving Results for Index 0: Umluj.
Could not find a hotel within 5000 meters
Retrieving Results for Index 1: Bahía Honda.
Could not find a hotel within 5000 meters
Retrieving Results for Index 2: Marsá Maţrūḩ.
Could not find a hotel within 5000 meters
Retrieving Results for Index 3: Yulara.
Could not find a hotel within 5000 meters
Retrieving Results for Index 4: Atar.
Could not find a hotel within 5000 meters
Retrieving Results for Index 5: Cabañas.
Could not find a hotel within 5000 meters
Retrieving Results for Index 6: Manama.
Could not find a hotel within 5000 meters
Retrieving Results for Index 7: Alice Springs.
Could not find a hotel within 5000 meters
--------------------n/Search Complete/n--------------------


In [104]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Umluj,SA,25.0213,37.2685,
1,Bahía Honda,CU,22.9036,-83.1592,
2,Marsá Maţrūḩ,EG,31.3525,27.2453,
3,Yulara,AU,-25.2406,130.9889,
4,Atar,MR,20.5169,-13.0499,
5,Cabañas,CU,22.9758,-82.9219,
6,Manama,BH,26.2154,50.5832,
7,Alice Springs,AU,-23.7000,133.8833,


In [106]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [108]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure

humidity_map.add_layer(markers)
humidity_map


Figure(layout=FigureLayout(height='420px'))